# Nepali GPT Experiment

First line of devkota's Bhikhari poem:

 हेर भिखारी अडि अडि आयो

In [6]:
text = "हेर भिखारी अडि अडि आयो"
print(text)
len(text)

हेर भिखारी अडि अडि आयो


22

If you count the number of characters in the above line of text, there are actually `15` user-perceived characters. But when we use the `len` method on that text, it returns `22`.

Non-english texts especially the ones that may make use of multiple Unicode codepoint to represent a single user-perceived character cannot be split using Python's built-in text splitting routines. They need special functions that are aware of graphemes.

Look at: https://github.com/alvinlindstam/grapheme/tree/master

Following has some good discussion on the grapheme cluster algorithm: https://bugs.python.org/issue30717

In [8]:
import grapheme

In [9]:
grapheme.length(text)

15

If you notice, `grapheme` module above correctly returns `15`.

In [ ]:
grapheme.